# Generate a Dynamic Graph
In this notebook, we generate a dynamic graph based on a synthetic LFR graph, where we define the tranformation, the speed and the parameters of the initial graph.
If your goal is to generate multiple graph copies for a statistical comparison of algorithms, consider instead the usage of `../experiments/01a_generate_graphs.py`.

In [31]:
import sys
sys.path.append('../')
from src.simulation import GraphMovementSimulation
import networkx as nx
from tqdm import tqdm
import os
import pandas as pd

Chose a transformation among those in the list: 
- 'birth', 
- 'fragment',
- 'split',
- 'merge',
- 'add_nodes',
- 'intermittence_nodes', 
- 'switch',
- 'break_up_communities',
- 'remove_nodes'

In [35]:
transformation='fragment'
TSS=50 # number of snapshots
TIME_TO_TRANFORM = 30 # time of the transformation: it corresponds to the number of snapshots needed to complete the transformation
assert TIME_TO_TRANFORM<TSS
SPEED=1/TIME_TO_TRANFORM

# select the initial number of nodes in the graph 
if transformation in ['remove_nodes','remove_edges']: 
    n=3000 
else: 
    n=1000


n_changes=3 # number of communities affected from the morphing transformations
mu = .2 # LFR mixing parameter: how much the communities are mixed between 0 and 1
        # You can also set the othe paramaters of the LFR benchmark in the initialization of GraphMovementSimulation

In [36]:
#generate the folder to store the data:
directory_path=f'../results/graphs/mu0{int(mu*10)}/{transformation}/'
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

In [37]:
gname=f'{transformation}_mu0{int(mu*10)}'
i=0
sim_count=0
while True:
    # sometimes the generation of the synthetic static graph fails due to the networkx implementation of the LFR static benchmark, so we need to repeat the generation till we find the right seed. 
    seed=i+sim_count
    try:
        sim = GraphMovementSimulation(n=n, mu=mu, gname=f'{gname}_{i}', seed=seed)
        G = sim.setup_transformation(transformation, n_splits=n_changes, save=True)
        break
    except:
        sim_count+=1
print(f'Graph #{i} -> Final seed: {seed}')


# Create the directory to store the graph:
if not os.path.exists(directory_path+ f'G{i:02}/'):
    # If it doesn't exist, create the directory
    os.makedirs(directory_path+ f'G{i:02}/')

# Store seed information
if os.path.exists(directory_path+'seeds.csv'):
    f=open(directory_path+'seeds.csv','a')
else:
    f=open(directory_path+'seeds.csv','w')
    f.write('num_graph,seed\n')
f.write(f'{i},{seed}\n')
f.close()



Graph #0 -> Final seed: 4


In [38]:
#run the transformation and 
_,GT=run=sim.run( G, N_Graph=i, delta=SPEED, transformation=transformation,timesteps=TSS,start_trans= 0, stop_trans= TSS)
pd.DataFrame({'y_true_init':GT['y_true_init'],'y_true_final':GT[ 'y_true_final'] }).fillna('')\
                .to_csv(f'../results/graphs/mu0{int(mu*10)}/{transformation}/G{i:02}/GT.csv.gz',sep=',',index=True,index_label='Node_id')



Processing fragment_mu02_0: 100%|██████████| 50/50 [00:09<00:00,  5.02it/s]
